In [1]:
import sys
sys.path.append('../')

import pandas as pd
from src.data.split_data import split_dataset
from src.models.train_model import BertClassifier

In [4]:
data = pd.read_csv('/home/nfrvnikita/projects/service4classification/data/raw/BRFRD.csv')
split_dataset(data=data)

In [5]:
data = pd.read_csv('/home/nfrvnikita/projects/service4classification/data/interim/train_set.csv')
model_path = 'cointegrated/rubert-tiny'
tokenizer_path = 'cointegrated/rubert-tiny'
bert_tiny = BertClassifier(model_path, tokenizer_path, data, epochs=4)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
bert_tiny.preparation()

/home/nfrvnikita/projects/service4classification/notebooks/../src/features/build_features.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['token_lens'] = token_lens
/home/nfrvnikita/projects/service4classification/notebooks/../src/features/build_features.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['token_lens'] = token_lens
/home/nfrvnikita/projects/service4classification/.venv/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW 

In [7]:
bert_tiny.fit()

100%|██████████| 870/870 [00:06<00:00, 142.77it/s]


Epochs: 1 | Train Loss:  0.202                 | Train Accuracy:  0.621                     | Val Loss:  0.196                         | Val Accuracy:  0.633


100%|██████████| 870/870 [00:06<00:00, 142.35it/s]


Epochs: 2 | Train Loss:  0.183                 | Train Accuracy:  0.663                     | Val Loss:  0.192                         | Val Accuracy:  0.640


100%|██████████| 870/870 [00:06<00:00, 143.55it/s]


Epochs: 3 | Train Loss:  0.163                 | Train Accuracy:  0.709                     | Val Loss:  0.199                         | Val Accuracy:  0.631


100%|██████████| 870/870 [00:06<00:00, 142.89it/s]

Epochs: 4 | Train Loss:  0.148                 | Train Accuracy:  0.746                     | Val Loss:  0.204                         | Val Accuracy:  0.630


(14717, 2918.980150837451)

In [12]:
test = pd.read_csv('/home/nfrvnikita/projects/service4classification/data/interim/test_set.csv')